In [ ]:
#Imports find_neighborhood function
#Credits: https://github.com/craigmbooth/chicago_neighborhood_finder, download necessary files
#Before you begin, Adjust file paths

import json
import os
from collections import namedtuple
import sys
from math import log,tan,pi,radians


Pt = namedtuple('Pt','x,y')
Edge = namedtuple('Edge','a,b')
Poly = namedtuple('Poly','name,edges')
_eps = 1e-5
_huge = sys.float_info.max
_tiny = sys.float_info.min

# NOTE - THIS FILE PATH MUST BE CHANGED
def load_json():
	file_in = open('/Users/er1cd/Downloads/Neighborhoods_2012_polygons.json', 'r')
	d = json.load(file_in)
	return d


def spherical_mercator_projection(longitude,latitude):
	#http://en.wikipedia.org/wiki/Mercator_projection <- invented in 1569!
	#http://stackoverflow.com/questions/4287780/detecting-whether-a-gps-coordinate-falls-within-a-polygon-on-a-map
	x = -longitude
	y = log(tan(radians(pi/4 + latitude/2)))
	return (x,y)


def rayintersectseg(p, edge):
	#http://rosettacode.org/wiki/Ray-casting_algorithm#Python
	#takes a point p=Pt() and an edge of two endpoints a,b=Pt() of a line segment returns boolean
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = Pt(p.x, p.y + _eps)
    intersect = False
 
    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False
 
    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        
        intersect = m_blue >= m_red
    return intersect
 
def is_odd(x): 
	return x%2 == 1

def ispointinside(p, poly):
    ln = len(poly)
    return is_odd(sum(rayintersectseg(p, edge)
                    for edge in poly.edges ))

def get_all_neighborhoods():
	d = load_json()
	shape_list=[]
	for shape_idx in range(len(d['features'])):
		name = d['features'][shape_idx]['properties']['SEC_NEIGH']

		edges =[]
		for coordinate_idx in range(len(d['features'][shape_idx]['geometry']['coordinates'][0])-1):
			lon_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][0]
			lat_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][1]
			
			lon_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][0]
			lat_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][1]
			
			x1,y1 = spherical_mercator_projection(lon_1,lat_1)
			x2,y2 = spherical_mercator_projection(lon_2,lat_2)
			edges.append(Edge(a=Pt(x=x1,y=y1),b=Pt(x=x2,y=y2)))
		
		shape_list.append(Poly(name=name,edges=tuple(edges)))
	return shape_list

def find_neighborhood(test_long,test_lat,all_neighborhoods):
	x,y = spherical_mercator_projection(test_long,test_lat)
	for neighborhood in all_neighborhoods:
		correct_neighborhood = ispointinside(Pt(x=x,y=y),neighborhood)
		if correct_neighborhood:
			return neighborhood.name

all_neighborhoods = get_all_neighborhoods()

test_long = -87.624069
test_lat = 41.862889

neighborhood = find_neighborhood(test_long,test_lat,all_neighborhoods)
#print(neighborhood), test

In [ ]:
#Sets path for Green_Roofs.csv
#NOTE: Adjust this before running code
path = '/Users/er1cd/Downloads/Green_Roofs.csv'


#Imports Chicago Neighborhood Boundaries in a GeoJson File
path2 = '/Users/er1cd/Downloads/NOI.geojson'

In [ ]:
#Import the Pandas Library
import pandas as pd

#Sets the file path of the City of Chicago's Green Roof Data file as a variable, G_roof
G_roof = path

#Define City of Chicago's Green Roof Data file as a variable, G_roof_data
G_roof_data = pd.read_csv(G_roof)

#Prints Green Roof Data as a checkpoint
print(G_roof_data)
G_roof_data.head()

#Create a subset of interest
Gsub = G_roof_data[['LOCATION','VEGETATED_SQFT','LONGITUDE','LATITUDE']]
#Gsub.head(), Checkpoint


In [ ]:
#Defines a function to sort Green Roof Data by neighborhood using Long. & Lat.
def sort_neighborhood(Gsub):
    test_long = Gsub['LONGITUDE']
    test_lat = Gsub['LATITUDE']
    Gsub['Neighborhood'] = find_neighborhood(test_long, test_lat ,all_neighborhoods)
    return Gsub

#Applies the function to all of Gsub, axis = 1
Gsub2 = Gsub.apply(sort_neighborhood, axis =1)

#Converts area from square feet to meters squared        
def SQFT_toM2(Gsub2):
    Gsub2['METERS_SQ'] = Gsub2['VEGETATED_SQFT']/10.764
    return Gsub2

#Applies the function, SQFT_toM2, to all of Gsub2
Gsub3 = Gsub2.apply(SQFT_toM2, axis =1)

#Groups green roofs by neighborhood, calculates the total area in M^2
Total_by_neighborhood = Gsub3.groupby('Neighborhood', as_index = False)['METERS_SQ'].sum()

#print(Total_by_neighborhood), checkpoint

#Sorts list of Chicago neighborhoods by total area of green roofs in descenfing order
Total_by_neighborhood2 = Total_by_neighborhood.sort_values('METERS_SQ', ascending = False, inplace = False)

print(Total_by_neighborhood2)

Gsub3.head()

In [ ]:
#Imports the folium library for easy map visualization
import folium

#Assigns Total_by_neighborhood2 to a variable, G_roof_data
G_roof_data = Total_by_neighborhood2
print(G_roof_data) #as a checkpoint

#Creates a map entered over Chicago, assigning it to the variable (m)
m = folium.Map(location =[41.8781, -87.6299], tiles = 'cartodbpositron', zoom_start = 12)


#Sets parameters for Choropleth showing greatest total area of green roofs (M^2)
folium.Choropleth(
    geo_data = path2,
    name = 'Green Roof Concentration',
    data = G_roof_data,
    columns = ['Neighborhood','METERS_SQ'],
    key_on = 'feature.properties.sec_neigh',
    fill_color = 'YlGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Green Roof Area (M^2)',
  ).add_to(m)

#Adds a layer control button to m
folium.LayerControl().add_to(m)

m